# 베이스라인 코드

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i

for df in [train, test]:
    df['지역'] = df['지역'].map(local_map)
    
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')
    
new_train = pd.DataFrame()
new_test = pd.DataFrame()

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '총임대세대수'] = temp['전용면적별세대수'].sum() # 베이스라인에서 추가한 내용
    new_train.loc[i, target] = temp.loc[0, target]
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
    new_test.loc[i, '총임대세대수'] = temp['전용면적별세대수'].sum() # 베이스라인에서 추가한 내용
    
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

x_train = new_train.drop(['단지코드',target], axis=1)
y_train = new_train[target]
x_test = new_test.drop(['단지코드'], axis=1)

model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(x_train, y_train)
pred = model.predict(x_test)

423it [00:06, 62.25it/s]
150it [00:02, 63.02it/s]


## 새로운 EDA (여기서부터 보시면 됩니다.)

In [3]:
# 예측결과 추가 및 피쳐 생성
import plotly.express as px
import plotly.graph_objects as go

new_test[target] = pred.astype(int)
new_train['type'] = 'train'
new_test['type'] = 'test'

data = pd.concat([new_train, new_test])
data['주차면수대비등록차량비율'] = data['등록차량수']/data['단지내주차면수']
data['주차면수대비총세대수비율'] = data['총세대수']/data['단지내주차면수']
data['거주율'] = 1- (data['공가수']/data['총세대수'])
data['거주율99%이상'] = data['거주율'].apply(lambda x: 1 if x>=0.95 else 0)

## 1. 등록차량 수 비교

In [4]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data[data['type']=='train']['등록차량수'], name='train', xbins={'size':50}))
fig.add_trace(go.Histogram(x=data[data['type']=='test']['등록차량수'], name='test', xbins={'size':50}))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

원래 train set의 `등록차량수` 분포와 randomforest 모델이 예측한 test set의 `등록차량수` 분포입니다. <br>
모양은 얼추 비슷해보이나, __train에 비해 test의 `등록차량수` 비중이 높은 곳도 있고, 낮은 곳도 보입니다.__

## 2. 분산 비교

In [5]:
print('train target의 분산:',data[data['type']=='train']['등록차량수'].std())
print('test target의 분산:',data[data['type']=='test']['등록차량수'].std())

train target의 분산: 391.03256288864696
test target의 분산: 300.67316577548064


train set에는 일부 __`등록차량수`가 압도적으로 높은 데이터 몇개가 있습니다__. train set의 분산이 더 높습니다.

## 3. 주차면수 대비 등록차량 비율 & 주차면수 대비 총 세대수 비율

### train

In [6]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data[data['type']=='train']['주차면수대비등록차량비율'], name='주차면수대비등록차량비율'))
fig.add_trace(go.Histogram(x=data[data['type']=='train']['주차면수대비총세대수비율'], name='주차면수대비총세대수비율', xbins={'size':0.1}))

fig.update_layout(barmode='overlay',bargap=0.0)
fig.update_traces(opacity=0.75)
fig.show()

총 세대수는 주차공간에 비해 매우 많아보입니다. 반면에 __`등록차량수`는 단지내 주차면수를 충족하지 못한 곳도 많아보이네요.__ <br>
아마 아직 입주가 많이 진행되지 않았거나, 차량이 없는 가정들도 많은 것 같습니다.

### test

In [7]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data[data['type']=='test']['주차면수대비등록차량비율'], name='주차면수대비등록차량비율', xbins={'size':0.1}))
fig.add_trace(go.Histogram(x=data[data['type']=='test']['주차면수대비총세대수비율'], name='주차면수대비총세대수비율', xbins={'size':0.1}))

fig.update_layout(barmode='overlay',bargap=0.0)
fig.update_traces(opacity=0.75)
fig.show()

베이스라인 RF모델로 예측한 결과는 __train set에 비해 `주차면수대비 등록차량비율`이 1이 넘어가는 경우가 많습니다.__ 기존 데이터 대비 과도하게 많이 예측하는 걸까요?

## 4. 거주율 대비 등록차량 비율 (train)

In [8]:
fig = px.scatter(
    data[data['type']=='train'], x='거주율', y='주차면수대비등록차량비율', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

미세하지만 회귀선에 따르면 `거주율`이 높을수록 `주차면수대비등록차량비율`도 높아보입니다. 그러나 단지내주차면수가 과도하게 많이 책정되어있거나, 차량이 없는 세대가 많은 단지라면 의미가 없을 수도 있습니다.

## 5. 그 외

여기부턴 여러 feature들의 train, test 비교 그래프입니다. RF가 대체로 train set 트렌드에 따라 잘 예측하고 있는 것 같지만, 그럼에도 __MAE가 100대라는 것은 test셋에 outlier가 많다는 의미일 수도 있겠습니다.__

In [9]:
fig = px.scatter(
    data, x='주차면수대비총세대수비율', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [10]:
fig = px.scatter(
    data, x='주차면수대비등록차량비율', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [11]:
fig = px.scatter(
    data, x='총세대수', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [12]:
fig = px.scatter(
    data, x='공가수', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [13]:
fig = px.scatter(
    data, x='지역', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [14]:
fig = px.scatter(
    data, x='총임대세대수', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [15]:
fig = px.scatter(
    data, x='지하철', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

In [16]:
fig = px.scatter(
    data, x='버스', y='등록차량수', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue',
    color='type', marginal_x='histogram', marginal_y='histogram'
)
fig.show()

## 결론 & 요약

- train set 데이터의 등록차량수 분포의 분산이 높다. 즉, outlier가 들어있을 가능성이 높다.
- RF모델은 train set의 분포에 비슷하게 예측은 하고 있지만, 그래도 MAE가 높다.
- 데이터가 좀 더 많았으면 좋겠다...